In [44]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from pathlib import Path

## Preprocessing the dataset

Images need to go to a df, then convert the RGB channels into pixels.

In [45]:
dataset_route = "../GroceryStoreDataset/dataset/"
train_txt_route = "../GroceryStoreDataset/dataset/train.txt"
val_txt_route = "../GroceryStoreDataset/dataset/val.txt"
test_txt_route = "../GroceryStoreDataset/dataset/test.txt"

In [46]:
def process_txt(route):
    df = pd.read_csv(route, sep=",",header=None,names=["route", "fine", "coarse", 'tensor'])
    return df

In [47]:
df = process_txt(train_txt_route)
df.head()

,route,fine,coarse,tensor
0,train/Fruit/Apple/Golden-Delicious/Golden-Deli...,0,0,NaN
1,train/Fruit/Apple/Golden-Delicious/Golden-Deli...,0,0,NaN
2,train/Fruit/Apple/Golden-Delicious/Golden-Deli...,0,0,NaN
3,train/Fruit/Apple/Golden-Delicious/Golden-Deli...,0,0,NaN
4,train/Fruit/Apple/Golden-Delicious/Golden-Deli...,0,0,NaN


In [48]:
df.shape

(2640, 4)

In [54]:
df_val = process_txt(val_txt_route)
df_val.head()

,route,fine,coarse,tensor
0,val/Fruit/Apple/Golden-Delicious/Golden-Delici...,0,0,NaN
1,val/Fruit/Apple/Golden-Delicious/Golden-Delici...,0,0,NaN
2,val/Fruit/Apple/Golden-Delicious/Golden-Delici...,0,0,NaN
3,val/Fruit/Apple/Golden-Delicious/Golden-Delici...,0,0,NaN
4,val/Fruit/Apple/Golden-Delicious/Golden-Delici...,0,0,NaN


In [56]:
df_test = process_txt(test_txt_route)
df_test.head()

,route,fine,coarse,tensor
0,test/Fruit/Apple/Golden-Delicious/Golden-Delic...,0,0,NaN
1,test/Fruit/Apple/Golden-Delicious/Golden-Delic...,0,0,NaN
2,test/Fruit/Apple/Golden-Delicious/Golden-Delic...,0,0,NaN
3,test/Fruit/Apple/Golden-Delicious/Golden-Delic...,0,0,NaN
4,test/Fruit/Apple/Golden-Delicious/Golden-Delic...,0,0,NaN


## From images to pixels

ResNet CNN uses a 224x224 pixels. Then, we must:

1. resize the images
2. convert them into a processable data for the cnn

In [49]:
def resize_image(route, size=(224,224)):
    img = tf.io.read_file(route)
    img = tf.image.decode_jpeg(img, channels=3)
    img_resized = tf.image.resize(img, [224, 224])
    img_normalised = img_resized / 255 # normalise data to improve performance and acc
    return img_normalised

In [50]:
df["tensor"] = df["route"].apply(lambda x: resize_image(os.path.join(dataset_route, x)))
df.head()

,route,fine,coarse,tensor
0,train/Fruit/Apple/Golden-Delicious/Golden-Deli...,0,0,"(((tf.Tensor(0.07058824, shape=(), dtype=float..."
1,train/Fruit/Apple/Golden-Delicious/Golden-Deli...,0,0,"(((tf.Tensor(0.5783479, shape=(), dtype=float3..."
2,train/Fruit/Apple/Golden-Delicious/Golden-Deli...,0,0,"(((tf.Tensor(0.34684873, shape=(), dtype=float..."
3,train/Fruit/Apple/Golden-Delicious/Golden-Deli...,0,0,"(((tf.Tensor(0.74612814, shape=(), dtype=float..."
4,train/Fruit/Apple/Golden-Delicious/Golden-Deli...,0,0,"(((tf.Tensor(0.394208, shape=(), dtype=float32..."


In [55]:
df_val["tensor"] = df_val["route"].apply(lambda x: resize_image(os.path.join(dataset_route, x)))
df_val.head()

,route,fine,coarse,tensor
0,val/Fruit/Apple/Golden-Delicious/Golden-Delici...,0,0,"(((tf.Tensor(0.44126683, shape=(), dtype=float..."
1,val/Fruit/Apple/Golden-Delicious/Golden-Delici...,0,0,"(((tf.Tensor(0.13513376, shape=(), dtype=float..."
2,val/Fruit/Apple/Golden-Delicious/Golden-Delici...,0,0,"(((tf.Tensor(0.15215617, shape=(), dtype=float..."
3,val/Fruit/Apple/Golden-Delicious/Golden-Delici...,0,0,"(((tf.Tensor(0.056168094, shape=(), dtype=floa..."
4,val/Fruit/Apple/Golden-Delicious/Golden-Delici...,0,0,"(((tf.Tensor(0.6362045, shape=(), dtype=float3..."


In [57]:
df_test["tensor"] = df_test["route"].apply(lambda x: resize_image(os.path.join(dataset_route, x)))
df_test.head()

,route,fine,coarse,tensor
0,test/Fruit/Apple/Golden-Delicious/Golden-Delic...,0,0,"(((tf.Tensor(0.5455182, shape=(), dtype=float3..."
1,test/Fruit/Apple/Golden-Delicious/Golden-Delic...,0,0,"(((tf.Tensor(0.5507003, shape=(), dtype=float3..."
2,test/Fruit/Apple/Golden-Delicious/Golden-Delic...,0,0,"(((tf.Tensor(0.2863658, shape=(), dtype=float3..."
3,test/Fruit/Apple/Golden-Delicious/Golden-Delic...,0,0,"(((tf.Tensor(0.42406806, shape=(), dtype=float..."
4,test/Fruit/Apple/Golden-Delicious/Golden-Delic...,0,0,"(((tf.Tensor(0.29791948, shape=(), dtype=float..."


## ResNet-34

[Link](https://www.analyticsvidhya.com/blog/2021/08/how-to-code-your-resnet-from-scratch-in-tensorflow/)

In [61]:
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [60]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [62]:
def ResNet34(shape = (224, 224, 3), classes = 81):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [65]:
model = ResNet34()
model.summary()

Model: "ResNet34"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_3    │ (None, 230, 230,  │          0 │ input_layer_3[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_79 (Conv2D)  │ (None, 115, 115,  │      9,472 │ zero_padding2d_3… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 115, 115,  │        256 │ conv2d_79[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_73       │ (None, 115, 115,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 58, 58,    │          0 │ activation_73[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_80 (Conv2D)  │ (None, 58, 58,    │     36,928 │ max_pooling2d_3[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 58, 58,    │        256 │ conv2d_80[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_74       │ (None, 58, 58,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_81 (Conv2D)  │ (None, 58, 58,    │     36,928 │ activation_74[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 58, 58,    │        256 │ conv2d_81[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_35 (Add)        │ (None, 58, 58,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_75       │ (None, 58, 58,    │          0 │ add_35[0][0]      │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_82 (Conv2D)  │ (None, 58, 58,    │     36,928 │ activation_75[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 58, 58,    │        256 │ conv2d_82[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_76       │ (None, 58, 58,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_83 (Conv2D)  │ (None, 58, 58,    │     36,928 │ activation_76[0]

 Total params: 25,542,993 (97.44 MB)

 Trainable params: 25,527,761 (97.38 MB)

 Non-trainable params: 15,232 (59.50 KB)

In [68]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

X_train = np.stack(df["tensor"].values)
y_train = np.array(df["fine"].values)

X_val = np.stack(df_val["tensor"].values)
y_val = np.array(df_val["fine"].values)

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
28/83 ━━━━━━━━━━━━━━━━━━━━ 3:15 4s/step - accuracy: 0.0313 - loss: 14.2722

KeyboardInterrupt: 